# Module 6: Analytics with Spark

# Introduction

This module builds on the previous Spark modules with a focus on analytics.

# Learning Outcomes

In this module, you will build skills in:

* Machine learning with Spark’s machine learning library (MLlib) 
* Avoiding operations that will be slow on large datasets


# Readings and Resources

We invite you to supplement this notebook with the following:

* Machine Learning Library (MLlib) Guide: https://spark.apache.org/docs/latest/ml-guide.html


<h1>Table of Contents<span class="tocSkip"></span></h1>
<br>
<div class="toc">
<ul class="toc-item">
<li><span><a href="#Module-6:-Analytics-with-Spark" data-toc-modified-id="Module-6:-Spark-with-Spark">Module 6: Analytics with Spark</a></span>
</li>
<li><span><a href="#Introduction" data-toc-modified-id="Introduction">Introduction</a></span>
</li>
<li><span><a href="#Learning-Outcomes" data-toc-modified-id="Learning-Outcomes">Learning Outcomes</a></span>
</li>
<li><span><a href="#Readings-and-Resources" data-toc-modified-id="Readings-and-Resources">Readings and Resources</a></span>
</li>
<li><span><a href="#Table-of-Contents" data-toc-modified-id="Table-of-Contents">Table of Contents</a></span>
</li>
<li><span><a href="#Spark-Transformations-and-Actions" data-toc-modified-id="Spark-Transformations-and-Actions">Spark Transformations and Actions</a></span>
<ul class="toc-item">
<li><span><a href="#Spark-Transformations" data-toc-modified-id="Spark-Transformations">Spark Transformations</a></span>
</li>
<li><span><a href="#Spark-Actions" data-toc-modified-id="Spark-Actions">Spark Actions</a></span>
</li>
</ul>
</li>
<li><span><a href="#Machine-Learning-with-Spark" data-toc-modified-id="Machine-Learning-with-Spark">Machine Learning with Spark</a></span>
<ul class="toc-item">
<li><span><a href="#MLlib-concepts" data-toc-modified-id="MLlib-concepts">MLlib concepts</a></span>
</li>
</ul>
<li><span><a href="#General-tips-for-writing-efficient-Spark-programs" data-toc-modified-id="General-tips-for-writing-efficient-Spark-programs">General tips for writing efficient Spark programs</a></span>
</li>
</li>
<li><span><a href="#References" data-toc-modified-id="References">References</a></span>
</li>
</ul>
</div>

# Spark Transformations and Actions

Spark **transformations** create a new dataset from an existing one.  The existing dataset could be a file from which you read data or a dataset that you have already transformed in a prior step.  All transformations in Spark are **lazy** &mdash; i.e. they do not compute their results right away. Instead, they remember the transformations applied to some base dataset but don't apply them until necessary. This accomplishes the following:

- It allows optimization of the required calculations &mdash; the scheduler may recognize that the set of transformations can be done more efficiently in a different order.  For example, a *filter* after a *map* will be slower than filtering first, (assuming the filter doesn't require the mapping to be done first).  The filter will reduce the amount of work the map needs to do.


- It helps with data recovery if data is lost part way through a computation.  Spark avoids storing intermediate results to disk so if a disk or node fails, the intermediate results won't be available on disk like they are with vanilla MapReduce.  Remembering the steps that were followed to create partial results allow the data to be reconstructed if necessary to continue a failed computation.

A transformation is not executed until an action follows it. An **action** runs the tasks that precede it in the DAG and typically aggregates the results of the transformations.

Let's look at some examples.


## Spark Transformations

Here is an example. The first line defines a base RDD from an external file. This dataset is not loaded in memory or otherwise acted on &mdash; `lines` is merely a pointer to the file. The second line defines `lineLengths` as the result of a `map` transformation. Again, `lineLengths` is not immediately computed, due to laziness. Finally, we run `reduce`, which is an action. At this point Spark breaks the computation into tasks to run on separate machines, and each machine runs both its part of the map and a local reduction, returning only its answer to the driver program.


    lines = sc.textFile("data.txt")
    lineLengths = lines.map(lambda s: len(s))
    totalLength = lineLengths.reduce(lambda a, b: a + b)

If we want to use `lineLengths` again later, we can add the following before the reduce, which would cause `lineLengths` to be saved in memory the first time it is computed:
    
    lineLengths.persist()
    

### map and flatMap

The two most import kinds of transformations that are used in Spark are `map` and `flatMap`.

The `map` and `flatMap` transformations are similar, in the sense that they take a line from the input RDD and apply a function to it. The way they differ is that the function in `map` returns only one element, while the function in `flatMap` can return a list of elements (0 or more) which can be iterated over.

More formally:

- **`map(func)`**:  Returns a new distributed dataset formed by passing each element of the source through a function you provide (`func`). Each element in the dataset is sent into the function, one at a time, and produce one element an output.


- **`flatMap(func)`**: Similar to map, but each input item can be mapped to 0 or more output items (so `func` should return a sequence or list rather than a single item). Each element goes into `func`, 0 or more elements (a collection) come out.


**Example 1:** 

    sc.parallelize([3,4,5]).map(lambda x: range(1,x)).collect() 
       
**NOTE**: `collect`: collect returns the elements of the RDD back to the driver program.

**Output:** Flattened out in a single list, 
[1, 2, 1, 2, 3, 1, 2, 3, 4] 


**Example 2:**

    sc.parallelize([3,4,5]).map(lambda x: [x,  x*x]).collect() 

**Output:**
[[3, 9], [4, 16], [5, 25]]

    sc.parallelize([3,4,5]).flatMap(lambda x: [x, x*x]).collect() 

**Output:** A flattened list, 
[3, 9, 4, 16, 5, 25]



**Example 3:**

Consider a file greetings.txt in HDFS with the following lines: 

Good Morning 
Good Evening 
Good Day 
Happy Birthday 
Happy New Year

    lines = sc.textFile("greetings.txt")
    lines.map(lambda line: line.split()).collect()

**Output:**
[['Good', 'Morning'], ['Good', 'Evening'], ['Good', 'Day'], ['Happy', 'Birthday'], ['Happy', 'New', 'Year']]


    lines.flatMap(lambda line: line.split()).collect()

**Output:**
['Good', 'Morning', 'Good', 'Evening', 'Good', 'Day', 'Happy', 'Birthday', 'Happy', 'New', 'Year']


We can do a word count of the file using `flatMap`:
    
    lines = sc.textFile("greetings.txt")
    sorted(lines.flatMap(lambda line: line.split()).map(lambda w: (w,1)).reduceByKey(lambda v1, v2: v1+v2).collect())

**Output:** 
[('Birthday', 1), ('Day', 1), ('Evening', 1), ('Good', 3), ('Happy', 2), ('Morning', 1), ('New', 1), ('Year', 1)]


**Example 4:**

Below is an example demonstrating the difference between the `map` and `flatMap` operations in an RDD using the Scala interactive shell. Here a `flatMap` flattens multiple arrays into one single array. We have a flatfile called "words.txt" which contains lines of words, separated by spaces.

    ~/user$ cat words.txt 
          
          line1 word1
          line2 word2 word1 
          line3 word3 word4
          line4 word1


    scala> val lines = sc.textFile("words.txt");
    scala> lines.map(_.split(" ")).take(3)

    res4: Array[Array[String]] = Array(Array(line1, word1), Array(line2, word2, word1), Array(line3, word3, word4))

The following flattens multiple lists into one single list:

    scala> lines.flatMap(_.split(" ")).take(3)
    res5: Array[String] = Array(line1, word1, line2)

### Full list of Spark Transformations

Both `map` and `flatMap` are very generic. There is a wide array of other more specialized transformations available in Spark. Follow this link for the full current list: https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations.

## Spark Actions

Actions trigger any previously constructed Spark transformations in the Spark directed acyclic graph to be evaluated.

**Example 1:**

`reduce`: Aggregate the elements of a dataset through a function

    names1 = sc.parallelize(["abe", "abby", "apple"])
    names1.reduce(lambda t1, t2: t1+t2)

**Output:**
abeabbyapple


    names2 = sc.parallelize(["apple", "beatty", "beatrice"]).map(lambda a: [a, len(a)])
    names2.collect()

**Output:**
[['apple', 5], ['beatty', 6], ['beatrice', 8]]

    names2.flatMap(lambda t: [t[1]]).reduce(lambda t1, t2: t1+t2)

**Output:**
19

**Example 2:**

    sc.parallelize([1,2,3]).flatMap(lambda x: [x,x,x]).collect()

**Output:**
[1, 1, 1, 2, 2, 2, 3, 3, 3]

**Example 3:**

`count`: Number of elements in the RDD

    names1 = sc.parallelize(["abe", "abby", "apple"])
    names1.count()

**Output:**
3

**Example 4:**

    saveAsTextFile(path, compressionCodecClass=None): Save RDD as text file, using string representations of elements.

Parameters:	

- `path`: path to file
- `compressionCodecClass`: (None by default) string i.e. “org.apache.hadoop.io.compress.GzipCodec”


    hockeyTeams = sc.parallelize(("wild", "blackhawks", "red wings", "wild", "oilers", "whalers", "jets", "wild"))
    hockeyTeams.map(lambda k: (k,1)).countByKey().items()
    hockeyTeams.saveAsTextFile("hockey_teams.txt")

**Output:**

    $ ls hockey_teams.txt/
    _SUCCESS	part-00001	part-00003	part-00005	part-00007
    part-00000	part-00002	part-00004	part-00006

Each partition is written to its own file.  There are 8 partitions in the dataset example here.

### Full list of Spark Actions

For your reference, this link will take you to the Spark actions and what they do: https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions.

# Machine Learning with Spark

As we learned in an earlier module, MLlib is Spark’s Machine Learning (ML) library. Its goal is to make practical machine learning scalable and easy. At a high level, it provides tools such as:

- **ML Algorithms**: Common learning algorithms such as classification, regression, clustering, and collaborative filtering


- **Featurization**: Feature extraction, transformation, dimensionality reduction, and selection


- **Pipelines**: Tools for constructing, evaluating, and tuning ML pipelines


- **Persistence**: Tools for saving and loading algorithms, models, and pipelines


- **Utilities**: Tools for linear algebra, statistics, data handling, etc.

## MLlib concepts

MLlib standardizes APIs for machine learning algorithms to make it easier to combine multiple algorithms into a single pipeline, or workflow. Here are some terms you will need to be familiar with:

- **DataFrame**: This ML API uses DataFrames from Spark SQL as an ML dataset, which can hold a variety of data types (e.g. a DataFrame could have different columns storing text, feature vectors, true labels, and predictions)


- **Transformer**: A transformer is an algorithm which can transform one DataFrame into another DataFrame


- **Estimator**: An estimator is an algorithm which can be fit on a DataFrame to produce a transformer


- **Pipeline**: A pipeline chains multiple transformers and estimators together to specify an ML workflow


- **Parameter**: All transformers and estimators now share a common API for specifying parameters

**Exercise**

The notebook below shows the full code for exploring two datasets and will introduce linear regression using MLlib. You will also do an exercise on the same topic. Please follow the link, and complete the exercise:

- https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/1896577977217881/932322184146507/6173011382478750/latest.html

# General tips for writing efficient Spark programs

- **When using join**: "Join" is one of the most expensive operations you will use in Spark, so it is worth doing what you can to shrink your data before performing a join:<br><br>
    - Consider data partitioning, shuffling, broadcasting, caching
    - Shuffle less often if you can
    - Shuffle better if you can


- **Partition the data**: This is one of the most common yet ignored issues. Suppose you partition some user data by country. A country with more users will take more time to process. The lucky nodes that get assigned countries with few users will complete their task quickly but will lie idle while the larger partitions finish processing. A better approach might be to split by user ID range. This should create more balanced partitions, and we can make the partitions smaller which would allow them to be processed simultaneously by a larger number of executors.


- **Never use `.collect()`**: Collect brings all the data from the executors to the driver, causes a lot of network load, and may even cause the driver to crash if the data is too large for the driver’s memory. You should only use `collect` for printing a subset of the data for debugging. Otherwise, refrain from using it.


* **Don’t use `GroupByKey`**: `GroupByKey` is used for collecting data with respect to a key. `GroupByKey` shuffles/redistributes all data to their respective partitions before merging them. This leads to a lot of network I/O. It is an expensive operation as it shuffles data from multiple partitions and creates another RDD. This operation is costly as it doesn't use a combiner local to a partition to reduce the data transfer. Hence, we do not recommend its use when you need to further aggregate grouped data. A better approach is to use `ReduceByKey`. It first groups data on each partition and then shuffles the data to respective partitions. This reduces network traffic.

**End of Module**

You have reached the end of this module.

If you have any questions, please reach out to your peers using the discussion boards. If you
and your peers are unable to come to a suitable conclusion, do not hesitate to reach out to
your instructor on the designated discussion board.

When you are comfortable with the content, you may proceed to the next module.

# References

* Spark MLlib documentation.  Retrieved from: https://spark.apache.org/docs/latest/ml-guide.html


* Steps involved in Supervised learning, retrieved from http://sebastianraschka.com/Articles/2014_intro_supervised_learning.html


* Submitting spark jobs to cluster.  Retrieved from Spark's official documentations https://spark.apache.org/docs/latest/submitting-applications.html


* Running pyspark as Spark Standalone job.  Retrieved from Anaconda documentation: https://docs.anaconda.com/anaconda-scale/howto/spark-basic/


* Article on best practices of writing production grade pyspark jobs.  Retrieved from Medium: https://developerzen.com/best-practices-writing-production-grade-pyspark-jobs-cb688ac4d20f


* Tutorial on automation using makefiles.  Retrieved from : https://swcarpentry.github.io/make-novice/02-makefiles/


* pip user guide.  Retrieved from: https://pip.pypa.io/en/stable/user_guide/


* Open source examples. Retrieved from Github: https://github.com/malexer/pytest-spark, https://github.com/ekampf/PySpark-Boilerplate, https://github.com/kawadia/pyspark.test